In [1]:
from __future__ import print_function
import numpy as np
import os
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
from train.shy_provider import *
from models.shy_frustrum_pointnet import *
ROOT_DIR = os.path.abspath('')
OUT_DIR = os.path.join(ROOT_DIR, 'outputs')
TEST_LOG = os.path.join(OUT_DIR, 'test')
TRAIN_LOG = os.path.join(OUT_DIR, 'train')
print(torch.cuda.is_available())

/home/ubuntu/shy-frustrum-pointnets/train /home/ubuntu/shy-frustrum-pointnets
True


In [2]:
EPOCH_CNT = 0
BATCH_SIZE = 32
NUM_POINT = 2048
MAX_EPOCH = 201
BASE_LEARNING_RATE = 0.001
# GPU_INDEX = FLAGS.gpu
# MOMENTUM = FLAGS.momentum
OPTIMIZER = 'adam'
NUM_CHANNEL =  4 # point feature channel
NUM_CLASSES = 2 # segmentation has two classes

MODEL_PATH = os.path.join(OUT_DIR, "model_120.pth")

In [3]:
# dataset preparation

train_dataset = FrustumDataset(npoints=NUM_POINT, split='train',
        rotate_to_center=True, random_flip=True, random_shift=True, one_hot = True)
test_dataset = FrustumDataset(npoints=NUM_POINT, split='val',
    rotate_to_center=True, one_hot=True)

In [4]:
train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=int(4))

test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=int(4))

In [5]:
# Creating and loading Frustrum_pointnet
in_dims = 4
net = FrustrumPointNent_v1(in_dims)
if torch.cuda.is_available():
    net.cuda()
    
net.load_state_dict(torch.load(MODEL_PATH))
print("Loaded '{}'.".format(MODEL_PATH))

Loaded '/home/ubuntu/shy-frustrum-pointnets/outputs/model_120.pth'.


In [6]:
def run_test(data, net, writer=None):
    batch_data, batch_label, batch_center, batch_heading_class, batch_heading_residual, \
    batch_size_class, batch_size_residual, batch_rot_angle, batch_one_hot_vec = data
    
    batch_data = batch_data.float().cuda()
    batch_one_hot_vec = batch_one_hot_vec.float().cuda()
    
    batch_label = batch_label.long()
    batch_center = batch_center.float()
    batch_heading_class = batch_heading_class.long()
    batch_heading_residual = batch_heading_residual.float()
    batch_size_class = batch_size_class.long()
    batch_size_residual = batch_size_residual.float()
    batch_rot_angle = batch_rot_angle.float()
    
    with torch.no_grad():
        end_points = net.eval()(batch_data, batch_one_hot_vec)

        loss = get_loss(batch_label, batch_center, batch_heading_class, \
                        batch_heading_residual, batch_size_class, batch_size_residual, end_points, None, None)
    
    
        iou2ds, iou3ds = compute_box3d_iou(end_points['center'].detach().numpy(), end_points['heading_scores'].detach().numpy(), end_points['heading_residuals'].detach().numpy(), \
                          end_points['size_scores'].detach().numpy(), end_points['size_residuals'].detach().numpy(), \
                          batch_center.numpy(), batch_heading_class.numpy(), batch_heading_residual.numpy(), batch_size_class.numpy(), batch_size_residual.numpy())
        correct = (torch.argmax(end_points['mask_logits'], 2) == batch_label )
        accuracy = torch.sum(correct.float()) / float(BATCH_SIZE*NUM_POINT)

    return loss.detach().numpy(), accuracy.numpy(), np.mean(iou2ds), np.mean(iou3ds)

In [7]:
def run_epoch_test(dataset, dataloader, train, epoch = None,
        writer=None):
    
    total_loss = 0.0
    total_acc = 0.0
    total_iou2ds = 0.0
    total_iou3ds = 0.0
    n_data = 0
    epoch_str = '' if epoch is None else '[Epoch {}/{}]'.format(
            str(epoch).zfill(len(str(MAX_EPOCH))), MAX_EPOCH)
    
    # Create a progress bar.
    pbar = tqdm(total=n_data, leave=False)
    
    mode = 'Train' if train else 'Test'
    
    
    for i, data in enumerate(dataloader):
        # Run one step.
        loss, seg_accuracy, mean_iou2ds, mean_iou3ds = run_test(data, net, writer) 
        
        

        batch_size = list(data[0].size())[0]
        total_loss += (loss * batch_size)
        total_acc += (seg_accuracy * batch_size)
        total_iou2ds += (mean_iou2ds * batch_size)
        total_iou3ds += (mean_iou3ds * batch_size)
    
        pbar.set_description('{} {} Loss: {:f}, SegAcc : {:.2f}%'.format(
            epoch_str, mode, loss, seg_accuracy))
        pbar.update(batch_size)
        n_data += batch_size

    
    pbar.close()
    mean_loss = total_loss / float(n_data)
    mean_acc = total_acc / float(n_data)
    mean_2d = total_iou2ds/float(n_data)
    mean_3d = total_iou3ds/float(n_data)
    
    if writer!= None:
        step = epoch * 300 + i
        writer.add_scalar('Loss/Test', loss, step)
        writer.add_scalar('SegmentationAccuracy/Test', seg_accuracy, step)
        writer.add_scalar('iou2d/Test', mean_iou2ds, step)
        writer.add_scalar('ioud3d/Test', mean_iou3ds, step)

    return mean_loss, mean_acc, mean_2d, mean_3d

In [9]:
epoch = 0
# testing on test and train datasets
mean_test_loss, mean_test_acc, mean_test_2d, mean_test_3d = run_epoch_test(
    test_dataset, test_dataloader, False, epoch, None)

mean_train_loss, mean_train_acc, mean_train_2d, mean_train_3d = run_epoch_test(
    train_dataset, train_dataloader, False, epoch, None)

log =""
log += 'Test Loss: {:f}, '.format(mean_test_loss)
log += 'Test SegmentationAcc: {:f}, '.format(mean_test_acc)
log += 'Test iou2d, iou3d: {:.2f}%, {:.2f}%, '.format(mean_test_2d, mean_test_3d)
print(log)
log =""
log += 'Train Loss: {:f}, '.format(mean_train_loss)
log += 'Train SegmentationAcc: {:.2f}%, '.format(mean_train_acc)
log += 'Train iou2d, iou3d: {:.2f}%, {:.2f}%, '.format(mean_train_2d, mean_train_3d)
print(log)

Test Loss: 5.481410, Test SegmentationAcc: 0.904816, Test iou2d, iou3d: 0.33%, 0.28%, 
Train Loss: 4.982156, Train SegmentationAcc: 0.93%, Train iou2d, iou3d: 0.36%, 0.31%, 
